In [ ]:
# importing necessary libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn 
from torchvision import datasets, transforms, models
from torchsampler import ImbalancedDatasetSampler
from IPython.display import clear_output
import cv2

In [ ]:
!pip install torchsampler
clear_output()

from torchsampler import ImbalancedDatasetSampler # used to balance the dataset

In [ ]:
# create a class object to transform each pixel to tensor, applying data augmentation, and finally normalizing each pixel values.
transforms = transforms.Compose([transforms.RandomApply([
                                      transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip()],0.7),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

In [ ]:
# mount colab to drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Loading train and validation datasets with ImageFolder. 
train_dataset = datasets.ImageFolder('/content/gdrive/MyDrive/object_detection/training_images', transform = transforms)
validation_dataset = datasets.ImageFolder('/content/gdrive/MyDrive/object_detection/validation_images', transform = transforms)

In [ ]:
# create training and validation batches with the given batch size and balance the train dataset using ImbalancedDatasetSampler. 
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, sampler = ImbalancedDatasetSampler(train_dataset), batch_size = batch_size, shuffle = False)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
# to check if the dataset is balanced 
labels_count = {0:0, 1:0}
for image, label in train_dataloader:
  for value in label:
    if value == 0:
      labels_count[0] +=1
    else:
      labels_count[1] += 1
  break
  print(labels_count)

In [ ]:
# to view a batch of objects and background 
image, label = next(iter(train_dataloader))
 
rows = int(image.shape[0]/8)
columns = int(image.shape[0]/rows)
plt.figure(figsize = (12, 8))

for i in range(image.shape[0]):
  plt.subplot(rows, columns, i + 1)
  plt.imshow(image[i].squeeze().permute(1, 2, 0))
  plt.title(label[i])
  plt.axis('off')
  plt.tight_layout()

In [ ]:
# defining the device 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
# defining resene18 model
model = models.resnet18(pretrained = True)

for param in model.parameters():
  param.requires_grad = False

# replacing the classifier layer of resnet model with our classification layers. 
classifier = nn.Sequential(nn.Linear(512, 512), 
                     nn.ReLU(), 
                     nn.Dropout(0.2),
                     nn.Linear(512, 256), 
                     nn.ReLU(), 
                     nn.Dropout(0.2), 
                     nn.Linear(256, 2)
                     )

model.fc = classifier
model.to(device)

clear_output()

In [ ]:
# defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
def train_loop(train_dataloader, model, criterion, optimizer):
  model.train()
  size = len(train_dataset)
  for batch, (image, label) in enumerate(train_dataloader):
    image = image.to(device)
    label = label.to(device)

    logits = model(image)
    optimizer.zero_grad()
    loss = criterion(logits, label)

    loss.backward()
    optimizer.step()

    loss += loss.item()
    if batch % 20 == 0:
      loss, current = loss.item(), batch * len(image)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
def validation_loop(dataloader, model, criterion, optimizer):
  model.eval()
  validation_loss = 0
  correct = 0
  min_validation_loss = np.inf
  
  with torch.no_grad():
    for images, labels in dataloader:
      images = images.to(device)
      labels = labels.to(device)
      pred = model(images)
      # print(pred)
      loss = criterion(pred, labels)
      # loss = loss.to(device)
      validation_loss += loss.item()
      correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
  
  validation_accuracy = validation_loss/len(dataloader)
  correct = correct/len(dataloader.dataset)
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
  if min_validation > validation_loss:
    print(f'Validation Loss Decreased({min_validation_loss:.6f}--->{validation_loss:.6f}) \t Saving The Model')
    min_validation_loss = validation_loss
    torch.save(model.state_dict(), '/content/gdrive/MyDrive/model1.pt')
    

In [ ]:
epoch = 10
for i in range(epoch):
  print(f'epoch {i+1}:' )
  train_loop(train_dataloader, model, criterion, optimizer)
  validation_loop(validation_dataloader, model, criterion, optimizer)


epoch 1:
loss: 1.386538  [    0/11309]
loss: 1.427075  [  640/11309]
loss: 1.403570  [ 1280/11309]
loss: 1.376046  [ 1920/11309]
loss: 1.372760  [ 2560/11309]
loss: 1.427269  [ 3200/11309]
loss: 1.311236  [ 3840/11309]
loss: 1.343429  [ 4480/11309]
loss: 1.315744  [ 5120/11309]
loss: 1.321029  [ 5760/11309]
loss: 1.312622  [ 6400/11309]
loss: 1.293927  [ 7040/11309]
loss: 1.308969  [ 7680/11309]
loss: 1.224994  [ 8320/11309]
loss: 1.327211  [ 8960/11309]
loss: 1.276870  [ 9600/11309]
loss: 1.287382  [10240/11309]
loss: 1.260913  [10880/11309]
Test Error: 
 Accuracy: 73.3%, Avg loss: 0.672634 

Validation Loss Decreased(inf--->39.036505) 	 Saving The Model
epoch 2:
loss: 1.253874  [    0/11309]
loss: 1.213523  [  640/11309]
loss: 1.221172  [ 1280/11309]
loss: 1.191337  [ 1920/11309]
loss: 1.182187  [ 2560/11309]
loss: 1.112442  [ 3200/11309]
loss: 1.306212  [ 3840/11309]
loss: 1.207225  [ 4480/11309]
loss: 1.054762  [ 5120/11309]
loss: 1.164393  [ 5760/11309]
loss: 1.037158  [ 6400/1130